In [1]:
%matplotlib inline

# 加载需要的库
import sys
sys.path.append("../")
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.datasets import DuolingoDataset

from aif360.datasets import StuperDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import  load_preproc_data_stuper
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_duolingo

from aif360.algorithms.inprocessing.adversarial_debiasing import FairEduNet

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html


#### 读取数据集和设置选项

In [2]:
# 获取数据集，进行训练集和测试集的划分
dataset_orig = load_preproc_data_duolingo()

privileged_groups = [{'ui_binary': 1}]
unprivileged_groups = [{'ui_binary': 0}]

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

In [3]:
# 打印出数据集的一些特征
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(734002, 7)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['ui_binary', 'learning_binary']


#### Privileged and unprivileged protected attribute values

[array([1.]), array([1.])] [array([0.]), array([0.])]


#### Dataset feature names

['ui_binary', 'learning_binary', 'delta', 'history_seen', 'history_correct', 'session_seen', 'is_workday=1']


#### 原始训练数据的指标

In [4]:
# 原始数据集的指标
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

#### Original training dataset

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.001677
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.001672


In [5]:
min_max_scaler = MaxAbsScaler()
dataset_orig_train.features = min_max_scaler.fit_transform(dataset_orig_train.features)
dataset_orig_test.features = min_max_scaler.transform(dataset_orig_test.features)
metric_scaled_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
# 缩放数据集 - 验证缩放是否不会影响组标签统计数据
display(Markdown("#### Scaled dataset - Verify that the scaling does not affect the group label statistics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_train.mean_difference())
metric_scaled_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_test.mean_difference())


#### Scaled dataset - Verify that the scaling does not affect the group label statistics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.001677
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.001672


### 学习计划分类器，无需去偏见

In [6]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to False
sess = tf.Session()
plain_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)

In [7]:
plain_model.fit(dataset_orig_train)

epoch 0; iter: 5733; batch classifier mean loss: 0.388449
epoch 1; iter: 5733; batch classifier mean loss: 0.384574
epoch 2; iter: 5733; batch classifier mean loss: 0.383689
epoch 3; iter: 5733; batch classifier mean loss: 0.383296
epoch 4; iter: 5733; batch classifier mean loss: 0.383145
epoch 5; iter: 5733; batch classifier mean loss: 0.382938
epoch 6; iter: 5733; batch classifier mean loss: 0.382795
epoch 7; iter: 5733; batch classifier mean loss: 0.382675
epoch 8; iter: 5733; batch classifier mean loss: 0.382558
epoch 9; iter: 5733; batch classifier mean loss: 0.382450
epoch 10; iter: 5733; batch classifier mean loss: 0.382368
epoch 11; iter: 5733; batch classifier mean loss: 0.382329
epoch 12; iter: 5733; batch classifier mean loss: 0.382280
epoch 13; iter: 5733; batch classifier mean loss: 0.382228
epoch 14; iter: 5733; batch classifier mean loss: 0.382183
epoch 15; iter: 5733; batch classifier mean loss: 0.382140
epoch 16; iter: 5733; batch classifier mean loss: 0.382138
epoch 1

In [8]:
# 将朴素模型应用于测试数据
dataset_nodebiasing_train = plain_model.predict(dataset_orig_train)
dataset_nodebiasing_test = plain_model.predict(dataset_orig_test)

In [9]:
# 朴素模型数据集的度量指标（未去偏见）
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(dataset_nodebiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(dataset_nodebiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

display(Markdown("#### Plain model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)


print("Test set: Statistical parity difference = %f" % classified_metric_nodebiasing_test.statistical_parity_difference())
print("Test set: Equalized Odds difference = %f" % classified_metric_nodebiasing_test.equalized_odds_difference())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())


#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.000165
Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.000228


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.866413
Test set: Statistical parity difference = 0.000228
Test set: Equalized Odds difference = 0.000036
Test set: Equal opportunity difference = 0.000000
Test set: Disparate impact = 1.000228


### 应用基于Adversarial Debiasing的处理中算法

In [10]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [11]:
# Learn parameters with debias set to True
# 在去偏见设置为True时学习参数
debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess)

In [12]:
debiased_model.fit(dataset_orig_train)

epoch 0; iter: 5733; batch classifier mean loss: 0.389401; batch adversarial mean loss: 0.676384
epoch 1; iter: 5733; batch classifier mean loss: 0.420244; batch adversarial mean loss: 0.675311
epoch 2; iter: 5733; batch classifier mean loss: 0.406580; batch adversarial mean loss: 0.675350
epoch 3; iter: 5733; batch classifier mean loss: 1.562821; batch adversarial mean loss: 0.655195
epoch 4; iter: 5733; batch classifier mean loss: 2.286305; batch adversarial mean loss: 0.612494
epoch 5; iter: 5733; batch classifier mean loss: 0.558253; batch adversarial mean loss: 0.675260
epoch 6; iter: 5733; batch classifier mean loss: 0.388230; batch adversarial mean loss: 0.674922
epoch 7; iter: 5733; batch classifier mean loss: 0.387553; batch adversarial mean loss: 0.674920
epoch 8; iter: 5733; batch classifier mean loss: 0.386589; batch adversarial mean loss: 0.674895
epoch 9; iter: 5733; batch classifier mean loss: 0.386087; batch adversarial mean loss: 0.674900
epoch 10; iter: 5733; batch cl

In [13]:
# 将朴素模型应用于测试数据
dataset_debiasing_train = debiased_model.predict(dataset_orig_train)
dataset_debiasing_test = debiased_model.predict(dataset_orig_test)

In [14]:
# 朴素模型数据集的度量指标（不去偏见）
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

# # 去偏见的模型数据集的度量指标
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())



display(Markdown("#### Plain model - without debiasing - classification metrics"))
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)

# print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
# print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
# print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
# print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
# print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())
print("Test set: Statistical parity difference = %f" % classified_metric_nodebiasing_test.statistical_parity_difference())
print("Test set: Equalized Odds difference = %f" % classified_metric_nodebiasing_test.equalized_odds_difference())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())


display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_debiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)

print("Test set: Statistical parity difference = %f" % classified_metric_debiasing_test.statistical_parity_difference())
print("Test set: Equalized Odds difference = %f" % classified_metric_debiasing_test.equalized_odds_difference())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.000165
Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.000228


#### Model - with debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.000748
Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.000827


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.866413
Test set: Statistical parity difference = 0.000228
Test set: Equalized Odds difference = 0.000036
Test set: Equal opportunity difference = 0.000000
Test set: Disparate impact = 1.000228


#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.866508
Test set: Statistical parity difference = 0.000827
Test set: Equalized Odds difference = 0.000245
Test set: Equal opportunity difference = 0.000000
Test set: Disparate impact = 1.000827
